In [4]:
import csv
import logging

# Configure logging
logging.basicConfig(filename='import_customer.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Read CSV file and return a list of dictionaries
def csv_to_dict_list(file_path):
    with open(file_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        data = [row for row in reader]
    return data

# Read CSV file and return a list of dictionaries
file_path = 'customer_import.csv'
logging.info('Reading from CSV file...')
print('Reading from CSV file...')
csv_data = csv_to_dict_list(file_path)
logging.info('CSV file read successfully.')
print('CSV file read successfully.')
logging.info('Importing Candidates Information from CSV file...')
print('Importing Candidates Information from CSV file...')


Reading from CSV file...
CSV file read successfully.
Importing Candidates Information from CSV file...


In [6]:
# Define the CandidateDTO class
class CandidateDTO:
    def __init__(self, candidate_id, first_name, last_name, email, phone, position, status):
        self.candidate_id = candidate_id
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.phone = phone
        self.position = position
        self.status = status

def csv_to_dict_list(file_path):
    with open(file_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        data = [row for row in reader]
    return data

# Example usage
file_path = 'customer_import.csv'
csv_data = csv_to_dict_list(file_path)

# Provided CSV data
data = csv_data

# Parse the CSV data and create CandidateDTO objects
candidates = []
for candidate_data in data:
    candidate = CandidateDTO(
        candidate_data['Candidate Id'],
        candidate_data['First Name'],
        candidate_data['Last Name'],
        candidate_data['Email'],
        candidate_data['Phone'],
        candidate_data['Position'],
        candidate_data['Status']
    )
    candidates.append(candidate)

# Print the CandidateDTO objects
for candidate in candidates:
    print(vars(candidate))  # Print object attributes as dictionary


{'candidate_id': '1', 'first_name': 'John', 'last_name': 'Doe', 'email': 'john.doe@email.com', 'phone': '123-456-7890', 'position': 'Data Architect', 'status': 'Pending'}
{'candidate_id': '2', 'first_name': 'Jane', 'last_name': 'Smith', 'email': 'jane.smith@email.com', 'phone': '987-654-3210', 'position': 'Data Architect', 'status': 'Interview Scheduled'}
{'candidate_id': '3', 'first_name': 'Michael', 'last_name': 'Jones', 'email': 'michael.jones@email.com', 'phone': '555-123-4567', 'position': 'Data Architect', 'status': 'Offer Extended'}
{'candidate_id': '4', 'first_name': 'Sarah', 'last_name': 'Clark', 'email': 'sarah.clark@email.com', 'phone': '111-222-3333', 'position': 'Data Architect', 'status': 'Rejected'}
{'candidate_id': '5', 'first_name': 'Chris', 'last_name': 'Nguyen', 'email': 'chris.nguyen@email.com', 'phone': '999-888-7777', 'position': 'Data Architect', 'status': 'Pending'}
{'candidate_id': '6', 'first_name': 'Emily', 'last_name': 'Kim', 'email': 'emily.kim@email.com', 

In [12]:
import json
import requests
import logging
import json
import csv
import random

# Configure logging
logging.basicConfig(filename='import_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Workable integration token (token)
token = '4pA5n-Ci0iA4o8u_DesrZpzkQxFxBDKkQ9id5klwmqw'

base_url = 'https://assignmentcorporate.workable.com/spi/v3/'

headers = {
    'accept': 'application/json',
    'content-type': 'application/json',
    'Authorization': f'Bearer {token}'
    }

# Define the CandidateDTO class
class CandidateDTO:
    def __init__(self, name, email, phone, address, job, domain, disqualified, sourced):
        self.name = name
        self.email = email
        self.phone = phone
        self.address = address
        self.job = job
        self.domain = domain
        self.disqualified = disqualified
        self.sourced = sourced # To prevent email notifications
    
    # Getter method for job
    def get_job(self):
        return self.job

    # Getter method for shortcode
    def get_shortcode(self):
        if 'shortcode' in self.job:
            return self.job['shortcode']
        else:
            return None
    def to_json(self):
        """
        Convert the CandidateDTO object to JSON format.
        """
        dto_dict = {
            "name": self.name,
            "email": self.email,
            "phone": self.phone,
            "address": self.address,
            "job": {
                "shortcode": self.job['shortcode'],
                "title": self.job['title']
            },
            "domain": self.domain,
            "disqualified": self.disqualified,
            "sourced": self.sourced
        }
        return json.dumps(dto_dict, indent=4)

class CandidateResponseDTO:
    def __init__(self, id, name, firstname, lastname, headline, image_url, account, job, stage,
                 disqualified, disqualified_at, disqualification_reason, hired_at, sourced,
                 profile_url, address, phone, email, outbound_mailbox, domain, uploader_id,
                 created_at, updated_at, cover_letter, summary, education_entries, experience_entries,
                 skills, answers, resume_url, tags, location, originating_candidate_id):
        self.id = id
        self.name = name
        self.firstname = firstname
        self.lastname = lastname
        self.headline = headline
        self.image_url = image_url
        self.account = account
        self.job = job
        self.stage = stage
        self.disqualified = disqualified
        self.disqualified_at = disqualified_at
        self.disqualification_reason = disqualification_reason
        self.hired_at = hired_at
        self.sourced = sourced
        self.profile_url = profile_url
        self.address = address
        self.phone = phone
        self.email = email
        self.outbound_mailbox = outbound_mailbox
        self.domain = domain
        self.uploader_id = uploader_id
        self.created_at = created_at
        self.updated_at = updated_at
        self.cover_letter = cover_letter
        self.summary = summary
        self.education_entries = education_entries
        self.experience_entries = experience_entries
        self.skills = skills
        self.answers = answers
        self.resume_url = resume_url
        self.tags = tags
        self.location = location
        self.originating_candidate_id = originating_candidate_id

class CandidateResponsePoolDTO:
    def __init__(self, id, name, firstname, lastname, headline, image_url, account, profile_url, address, phone, email, outbound_mailbox, domain,
                 created_at, updated_at, job, stage, disqualified, disqualified_at, disqualification_reason,
                 hired_at=None, sourced=False, uploader_id=None, cover_letter=None, summary=None, education_entries=[], experience_entries=[],
                 skills=[], answers=[], resume_url=None, tags=[], location={}, originating_candidate_id=None):
        self.id = id
        self.name = name
        self.firstname = firstname
        self.lastname = lastname
        self.headline = headline
        self.image_url = image_url
        self.account = account
        self.job = job
        self.stage = stage
        self.disqualified = disqualified
        self.disqualified_at = disqualified_at
        self.disqualification_reason = disqualification_reason
        self.hired_at = hired_at
        self.sourced = sourced
        self.profile_url = profile_url
        self.address = address
        self.phone = phone
        self.email = email
        self.outbound_mailbox = outbound_mailbox
        self.domain = domain
        self.uploader_id = uploader_id
        self.created_at = created_at
        self.updated_at = updated_at
        self.cover_letter = cover_letter
        self.summary = summary
        self.education_entries = education_entries
        self.experience_entries = experience_entries
        self.skills = skills
        self.answers = answers
        self.resume_url = resume_url
        self.tags = tags
        self.location = location
        self.originating_candidate_id = originating_candidate_id

class JobsDTO:
    def __init__(self, title, shortcode):
        self.title = title
        self.shortcode = shortcode

def csv_to_dict_list(file_path):
    with open(file_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        data = [row for row in reader]
    return data

# Example usage
file_path = 'customer_import.csv'
csv_data = csv_to_dict_list(file_path)
# Provided CSV data
data = csv_data

def generate_email_suffix(candidate_data):
    first_name = candidate_data['First Name'][0:1].lower()
    last_name = candidate_data['Last Name'][:2].lower()
    email = candidate_data['Email']
    random_integer = str(random.randint(100, 999))  # Generate a random integer between 100 and 999
    # Create the email format
    email_format = f"{first_name}{last_name}_{email[:-4]}{random_integer}{email[-4:]}"
    return email_format

def generate_full_address(candidate_data):
    address = candidate_data['Address']
    address_2 = candidate_data['Address 2']
    city = candidate_data['City']
    state = candidate_data['State']
    country = candidate_data['Country']
    zip_code = candidate_data['Zip']
    # Concatenate the address components
    full_address = ', '.join(filter(None, [address, address_2, city, state, country, zip_code]))
    return full_address

def is_disqualified(candidate_data):
    status = candidate_data['Status']
    # Check if the status is 'Rejected'
    disqualified = True if status == 'Rejected' else False
    return disqualified

def csv_to_dict_list(file_path):
    with open(file_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        data = [row for row in reader]
    return data

# Fetch job data
res = requests.get(base_url + 'jobs', headers=headers)
job_data = res.json()  # Parse JSON response

# Check if the response is successful
if res.status_code == 200:
    jobs = []
    for job in job_data['jobs']:
        # Create JobsDTO object for each job
        job_dto = JobsDTO(
            job['title'],
            job['shortcode'],
        )
        jobs.append(job_dto)

    # # Print the attributes of JobsDTO objects
    # for job in jobs:
    #     print(vars(job))  # Print object attributes as dictionary
else:
    print("Failed to fetch job data:", res.status_code)

def position(candidate_data, jobs):
    candidate_position = candidate_data['Position']
    for job in jobs:
        if candidate_position == job.title:
            return {
                "shortcode": job.shortcode,
                "title": job.title
            }
    return None
    
# Parse the CSV data and create CandidateDTO objects
candidates = []
for candidate_data in data:
    candidate = CandidateDTO(
        candidate_data['First Name'] + ' ' + candidate_data['Last Name'],
        generate_email_suffix(candidate_data),
        candidate_data['Phone'],
        generate_full_address(candidate_data),
        position(candidate_data, jobs),
        candidate_data['Referred By'],
        is_disqualified(candidate_data),
        sourced=False  # Set sourced to False to prevent email notifications THIS MEANS CANDIDATE APPLIED
    )
    candidates.append(candidate)

# # Print the CandidateDTO objects
# for candidate in candidates:
#     print(vars(candidate))  # Print object attributes as dictionary

# Convert CandidateDTO objects to dictionaries
candidate_dicts = [vars(candidate) for candidate in candidates]
# print(candidate_dicts)
# Serialize the list of dictionaries to JSON
candidates_json = json.dumps(candidate_dicts, indent=4)

# Print the JSON string
print(candidates_json)

[
    {
        "name": "John Doe",
        "email": "jdo_john.doe@email578.com",
        "phone": "123-456-7890",
        "address": "123 Main St, Anytown, CA, United States, 12345",
        "job": {
            "shortcode": "F8D1EA3849",
            "title": "Data Architect"
        },
        "domain": "Employee referral",
        "disqualified": false,
        "sourced": false
    },
    {
        "name": "Jane Smith",
        "email": "jsm_jane.smith@email257.com",
        "phone": "987-654-3210",
        "address": "456 Oak St, Someville, NY, United States, 56789",
        "job": {
            "shortcode": "F8D1EA3849",
            "title": "Data Architect"
        },
        "domain": "Job board",
        "disqualified": false,
        "sourced": false
    },
    {
        "name": "Michael Jones",
        "email": "mjo_michael.jones@email491.com",
        "phone": "555-123-4567",
        "address": "789 Pine St, Sometown, TX, United States, 67890",
        "job": {
            "

In [39]:
# Print the CandidateDTO objects
for candidate in candidates:
    print(vars(candidate))  # Print object attributes as dictionary

{'name': 'John Doe', 'email': 'jdo_john.doe@email.com', 'phone': '123-456-7890', 'address': '123 Main St, Anytown, CA, United States, 12345', 'job': {'shortcode': 'F8D1EA3849', 'title': 'Data Architect'}, 'domain': 'Employee referral', 'disqualified': False, 'sourced': False}
{'name': 'Jane Smith', 'email': 'jsm_jane.smith@email.com', 'phone': '987-654-3210', 'address': '456 Oak St, Someville, NY, United States, 56789', 'job': {'shortcode': 'F8D1EA3849', 'title': 'Data Architect'}, 'domain': 'Job board', 'disqualified': False, 'sourced': False}
{'name': 'Michael Jones', 'email': 'mjo_michael.jones@email.com', 'phone': '555-123-4567', 'address': '789 Pine St, Sometown, TX, United States, 67890', 'job': {'shortcode': 'F8D1EA3849', 'title': 'Data Architect'}, 'domain': 'Employee referral', 'disqualified': False, 'sourced': False}
{'name': 'Sarah Clark', 'email': 'scl_sarah.clark@email.com', 'phone': '111-222-3333', 'address': 'Metroville, CA, United States, 54321', 'job': {'shortcode': 'F

In [20]:
for candidate in candidate_dicts:
    print(f"Name: {candidate['name']}")
    print(f"Job: {candidate['job']['title']}")
    print("-" * 30)

Name: John Doe
Job: Data Architect
------------------------------
Name: Jane Smith
Job: Data Architect
------------------------------
Name: Michael Jones
Job: Data Architect
------------------------------
Name: Sarah Clark
Job: Data Architect
------------------------------
Name: Chris Nguyen
Job: Data Architect
------------------------------
Name: Emily Kim
Job: Data Architect
------------------------------
Name: Aaron Lee
Job: Data Architect
------------------------------
Name: Olivia Evans
Job: Data Architect
------------------------------
Name: James Miller
Job: Data Architect
------------------------------
Name: Lily Chen
Job: Data Architect
------------------------------
Name: David Johnson
Job: Data Architect
------------------------------
Name: Amy Williams
Job: Data Architect
------------------------------
Name: Robert Lee
Job: Data Architect
------------------------------
Name: Michelle Nguyen
Job: Data Architect
------------------------------
Name: Andrew Smith
Job: Data Arch

In [31]:
# Iterate through the list of CandidateDTO objects
for candidate in candidates:
    # Access the get_job() method for each candidate
    # job_shortcode = candidate.get_shortcode()
    # print("Job details for candidate", candidate.name, ":", job_shortcode)
    # type(job_shortcode)
    # job_details = candidate.get_job()
    # print("Job details for candidate", candidate.name, ":", job_details)
    shortcode = candidate.get_shortcode()
    if not shortcode:
        print("No shortcode found for candidate:", candidate['name'])
        continue
    url = f'{base_url}/jobs/{shortcode}/candidates' # Format the URL with the shortcode
    print("URL Template:", url)  # Print URL template for debugging


URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Template: https://assignmentcorporate.workable.com/spi/v3//jobs/F8D1EA3849/candidates
URL Templa

In [13]:
# Function to import candidates into the Talent Pool
def import_candidates(candidate_data):
    url = f'{base_url}/jobs/F8D1EA3849/candidates'  # Update the URL
    
    # Prepare payload with candidate data
    payload = {
        'candidates': candidate_data,
    }

    # Make POST request to import candidates
    response = requests.post(url, headers=headers, json=payload)

    # Check response status
    if response.status_code == 201:
        print('Candidates imported successfully!')
    else:
        print('Failed to import candidates.')
        print('Response:', response.text)
        print('Response status:', response.status_code)

import_candidates(candidates_json)

Failed to import candidates.
Response: {"error":"either name or firstname and lastname should be part of the candidate's payload"}
Response status: 422


In [8]:
responses = []        
def import_candidate(candidates_data):
    # importing shortcode in the POST url
    # shortcode = candidate.get_shortcode() # if i save shortcode
    # if not shortcode:
    #     print("No shortcode found for candidate:", candidate['name'])
    # print({candidate.name}) # access name
    # print({candidate.job["shortcode"]}) # access job[shortcode]
    url = f'{base_url}jobs/{candidate.job["shortcode"]}/candidates'
    print('url:', url)
    # print("URL Template:", url)  # Print URL template for debugging
    # print("Payload:", payload)  # Print payload for debugging
    # print(candidate_data)
    # Make POST request for the current candidate
    print("Importing candidate ",{candidate.name})
    # print(url)
    candidate_json = candidate.to_json()
    # print(candidate_json)
    # print(type(candidates_json))

    # Set headers for request
    headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token}'
    }

    # Set response parameteres
    response = requests.post(url, candidate_json, headers=headers)
    
    # Check response status
    if response.status_code == 201:
        print(f'Candidate {candidate.name} imported successfully!')
        
        # # Response as text
        # print('Response:', response.text)

        # Response as JSON
        response_json = json.loads(response.text)
        print(json.dumps(response_json, indent=4))
        
        # Check Remaining limit
        remaining_limit = int(response.headers.get('x-rate-limit-remaining', 0)) # If the key 'x-rate-limit-remaining' is not found in the dictionary, the get() method returns the default value, which is 0 in this case.
        print('Remaining limit:', remaining_limit)
        
        print(response.status_code)
        responses.append(response_json)
    else:
        print(f'Failed to import candidate {candidate.name}.')
        print('Response:', response.text)
        print('Response status:', response.status_code)

print("Importing candidates...")
for candidate in candidates:
    import_candidate(candidates)


Importing candidates...
url: https://assignmentcorporate.workable.com/spi/v3/jobs/F8D1EA3849/candidates
Importing candidate  {'John Doe'}
Candidate John Doe imported successfully!
{
    "status": "created",
    "candidate": {
        "id": "133e23fa",
        "name": "John Doe",
        "firstname": null,
        "lastname": null,
        "headline": null,
        "image_url": null,
        "account": {
            "subdomain": "assignmentcorporate",
            "name": "AssignmentCorporate"
        },
        "job": {
            "shortcode": "F8D1EA3849",
            "title": "Data Architect"
        },
        "stage": "Applied",
        "disqualified": false,
        "disqualified_at": null,
        "disqualification_reason": null,
        "hired_at": null,
        "sourced": false,
        "profile_url": "https://assignmentcorporate.workable.com/backend/jobs/3723929/candidates/322838312",
        "address": "123 Main St, Anytown, CA, United States, 12345",
        "phone": "123-45

In [16]:
def import_candidate(candidate, headers):
    responses = []

    # importing shortcode in the POST url
    url = f'{base_url}jobs/{candidate.job["shortcode"]}/candidates'
    
    candidate_json = candidate.to_json()
    
    # Set response parameters
    response = requests.post(url, candidate_json, headers=headers)
    
    # Check response status
    if response.status_code == 201:
        print(f'Candidate {candidate.name} imported successfully!')
        logging.info(f'Candidate {candidate.name} imported successfully!')
        
        # Response as JSON
        response_json = json.loads(response.text)
        print(json.dumps(response_json, indent=4))
        
        # Convert JSON response to DTO object
        candidate_response = CandidateResponseDTO.from_json(response_json)

        # Print response details
        print(f'Response ID: {candidate_response.id}')
        print(f'Response Name: {candidate_response.name}')
        
        logging.info('Response ID: %s', candidate_response.id)
        logging.info('Response Name: %s', candidate_response.name)

        responses.append(candidate_response)
    else:
        logging.error(f'Failed to import candidate {candidate.name}.')
        print(f'Failed to import candidate {candidate.name}.')
        print('Response:', response.text)
        logging.error('Response: %s', response.text)
        print('Response status:', response.status_code)
        logging.error('Response status: %d', response.status_code)

    return responses




















# Prompt the user to select the endpoint
print("Select an endpoint to send a POST request:")
print("1. jobs/{shortcode}/candidates")
print("2. /talent_pool/{stage}/candidates")

choice = input("Enter your choice (1 or 2): ")

if choice == '1':
    print("Importing candidates by POST ...")
    for candidate in candidates:
        import_candidate(candidates, headers)
# elif choice == '2':
#     stage = input("Enter the stage: ")
#     post_to_talent_pool_candidates(stage)
else:
    print("Invalid choice. Please select either 1 or 2.")

Select an endpoint to send a POST request:
1. jobs/{shortcode}/candidates
2. /talent_pool/{stage}/candidates
Invalid choice. Please select either 1 or 2.


In [160]:
for response in responses:
    # Access the 'candidate' object within each response
    candidate = response['candidate']

    # Access and print the 'name' and 'id' fields
    print("Name:", candidate['name'])
    print("ID:", candidate['id'])
    print()  # Add a blank line between each response


Name: {'status': 'created', 'candidate': {'id': '1339c968', 'name': 'John Doe', 'firstname': None, 'lastname': None, 'headline': None, 'image_url': None, 'account': {'subdomain': 'assignmentcorporate', 'name': 'AssignmentCorporate'}, 'job': {'shortcode': 'F8D1EA3849', 'title': 'Data Architect'}, 'stage': 'Applied', 'disqualified': False, 'disqualified_at': None, 'disqualification_reason': None, 'hired_at': None, 'sourced': False, 'profile_url': 'https://assignmentcorporate.workable.com/backend/jobs/3723929/candidates/322552982', 'address': '123 Main St, Anytown, CA, United States, 12345', 'phone': '123-456-7890', 'email': 'jdo_john.doe@email225.com', 'outbound_mailbox': '4e3elrgos11u@outbound.workablemail.com', 'domain': 'Employee referral', 'uploader_id': None, 'created_at': '2024-01-30T19:16:04Z', 'updated_at': '2024-01-30T19:16:04Z', 'cover_letter': None, 'summary': None, 'education_entries': [], 'experience_entries': [], 'skills': [], 'answers': [], 'resume_url': None, 'tags': [], 

In [ ]:
# Parse the JSON string into a Python dictionary
response_dict = json.loads(response_json)

# Extract the desired part
desired_part = json.dumps(response_dict['candidate'])

print(desired_part)

In [113]:
# Initialize an empty dictionary to store ID-Name pairs
id_name_dict = {}

# Iterate through each response
for response in responses:
    # Access the 'candidate' object within each response
    candidate = response['candidate']

    # Extract the 'name' and 'id' fields
    name = candidate['name']
    candidate_id = candidate['id']

    # Populate the dictionary with ID as keys and Name as values
    id_name_dict[candidate_id] = name

# Print the dictionary containing ID-Name pairs
print(id_name_dict)


{'1339c968': 'John Doe', '1339c969': 'Jane Smith', '1339c96b': 'Michael Jones', '1339c96e': 'Sarah Clark', '1339c971': 'Chris Nguyen', '1339c974': 'Emily Kim', '1339c976': 'Aaron Lee', '1339c97c': 'Olivia Evans', '1339c98e': 'James Miller', '1339c9a0': 'Lily Chen', '1339c9b5': 'David Johnson', '1339c9d0': 'Amy Williams', '1339c9e8': 'Robert Lee', '1339c9f2': 'Michelle Nguyen', '1339c9ff': 'Andrew Smith', '1339ca0e': 'Karen Johnson', '1339ca1e': 'Justin Kim', '1339ca29': 'Grace Li', '1339ca31': 'Christopher Brown', '1339ca3a': 'Isabella Wang', '1339ca45': 'Justin Johnson', '1339ca4b': 'Sophia Lee'}


In [21]:
# Iterate through the list of responses
for index, response in enumerate(responses, start=1):
    candidate = response['candidate']
    job = candidate['job']  # Access the job dictionary
    
    # Print candidate details with numbers next to names
    print(f"{index}. Name: {candidate['name']}, ID: {candidate['id']}, Job Title: {job['title']}, Shortcode: {job['shortcode']}")

# Ask the user to input a number corresponding to a candidate
selected_number = input("Enter the number corresponding to your choice: ")

# Validate the user input and retrieve the selected candidate details
try:
    selected_index = int(selected_number)
    if 1 <= selected_index <= len(responses):
        selected_candidate = responses[selected_index - 1]['candidate']
        selected_id = selected_candidate['id']
        selected_shortcode = selected_candidate['job']['shortcode']
        print(f"You selected Candidate ID: {selected_id}, Shortcode: {selected_shortcode}")
    else:
        print("Invalid input: Please enter a number within the given range.")
except ValueError:
    print("Invalid input: Please enter a valid number.")

id =selected_id
shortcode = selected_shortcode

headers = {
'accept': 'application/json',
'Content-Type': 'application/json',
'Authorization': f'Bearer {token}'
}

# Get request based on {id}
def get_candidate_info(base_url, id, token):
    
    url = f'{base_url}candidates/{id}' # url = f'{base_url}{shortcode}/candidates/{id}'
    print("URL:", url)
    try:
        # Send the GET request
        response = requests.get(url, headers=headers)
        
        # Check the response status code
        if response.status_code == 200:
            print("Response:")
            print(response.text)
        else:
            print(f"Error: {response.status_code} - {response.reason}")
    except requests.exceptions.RequestException as e:
        print("Error:", e)

# Get request from responses DTO based on {id}
def print_candidate_data_by_id(responses, desired_ids):
    # Iterate through each candidate dictionary in the responses list
    for candidate_info in responses:
        # Extract the "candidate" part from the candidate dictionary
        candidate_data = candidate_info.get('candidate')
        if candidate_data and candidate_data.get('id') in desired_ids:
            # Print the candidate data
            print({'candidate': candidate_data})

print("Choose an option:")
print("1. Perform a GET request using the API")
print("2. Print candidate data locally")
choice = input("Enter your choice (1 or 2): ")

if choice == '1':
    get_candidate_info(base_url, id, token)
elif choice == '2':
    print_candidate_data_by_id(responses, id)
else:
    print("Invalid choice. Please enter 1 or 2.")


1. Name: John Doe, ID: 133e23fa, Job Title: Data Architect, Shortcode: F8D1EA3849
2. Name: Jane Smith, ID: 133e23fd, Job Title: Data Architect, Shortcode: F8D1EA3849
3. Name: Michael Jones, ID: 133e23ff, Job Title: Data Architect, Shortcode: F8D1EA3849
4. Name: Sarah Clark, ID: 133e2400, Job Title: Data Architect, Shortcode: F8D1EA3849
5. Name: Chris Nguyen, ID: 133e2401, Job Title: Data Architect, Shortcode: F8D1EA3849
6. Name: Emily Kim, ID: 133e2406, Job Title: Data Architect, Shortcode: F8D1EA3849
7. Name: Aaron Lee, ID: 133e240c, Job Title: Data Architect, Shortcode: F8D1EA3849
8. Name: Olivia Evans, ID: 133e2417, Job Title: Data Architect, Shortcode: F8D1EA3849
9. Name: James Miller, ID: 133e2420, Job Title: Data Architect, Shortcode: F8D1EA3849
10. Name: Lily Chen, ID: 133e242a, Job Title: Data Architect, Shortcode: F8D1EA3849
11. Name: David Johnson, ID: 133e2438, Job Title: Data Architect, Shortcode: F8D1EA3849
12. Name: Amy Williams, ID: 133e2446, Job Title: Data Architect, S

In [175]:
# Initialize an empty dictionary to store ID-Name pairs
id_name_dict = {}

# Iterate through each response
for response in responses:
    # Access the 'candidate' object within each response
    candidate = response['candidate']

    # Extract the 'name' and 'id' fields
    name = candidate['name']
    candidate_id = candidate['id']

    # Populate the dictionary with ID as keys and Name as values
    id_name_dict[candidate_id] = name

# Print the dictionary containing ID-Name pairs
print(id_name_dict)

{'1339c968': 'John Doe', '1339c969': 'Jane Smith', '1339c96b': 'Michael Jones', '1339c96e': 'Sarah Clark', '1339c971': 'Chris Nguyen', '1339c974': 'Emily Kim', '1339c976': 'Aaron Lee', '1339c97c': 'Olivia Evans', '1339c98e': 'James Miller', '1339c9a0': 'Lily Chen', '1339c9b5': 'David Johnson', '1339c9d0': 'Amy Williams', '1339c9e8': 'Robert Lee', '1339c9f2': 'Michelle Nguyen', '1339c9ff': 'Andrew Smith', '1339ca0e': 'Karen Johnson', '1339ca1e': 'Justin Kim', '1339ca29': 'Grace Li', '1339ca31': 'Christopher Brown', '1339ca3a': 'Isabella Wang', '1339ca45': 'Justin Johnson', '1339ca4b': 'Sophia Lee'}


In [84]:
import requests
id =selected_id
shortcode = 'F8D1EA3849'

id_input = input("Please enter the ID you are looking for: ")
shortcode_input = input("Please enter the shortcode corresponding to the Job you are looking for: ")

url = f'{base_url}candidates/{id}'
print(url)
# {"subdomain":"assignmentcorporate","name":"AssignmentCorporate"}
# TO SWSTO ME TO SHORTCODE https://assignmentcorporate.workable.com/spi/v3/F8D1EA3849/candidates/id
# url = f'{base_url}{shortcode}/candidates/{id}'
print(url)
headers = {
'accept': 'application/json',
'Content-Type': 'application/json',
'Authorization': f'Bearer {token}'
}
response = requests.get(url, headers=headers)

print(response.text)

https://assignmentcorporate.workable.com/spi/v3/candidates/1338dd32
https://assignmentcorporate.workable.com/spi/v3/candidates/1338dd32


{"candidate":{"id":"1338dd32","name":"John DoeL","firstname":"John","lastname":"Doel","headline":null,"image_url":null,"account":{"subdomain":"assignmentcorporate","name":"AssignmentCorporate"},"job":{"shortcode":"F8D1EA3849","title":"Data Architect"},"stage":"Applied","disqualified":false,"disqualified_at":null,"disqualification_reason":null,"hired_at":null,"sourced":false,"profile_url":"https://assignmentcorporate.workable.com/backend/jobs/3723929/candidates/322492512","address":"123 Main St, Anytown, CA, United States, 12345","phone":"123-456-7890","email":"jdo_john.doe@email22.com","outbound_mailbox":"4e3e2eq56bx1@outbound.workablemail.com","domain":"Employee referral","uploader_id":null,"created_at":"2024-01-30T16:08:28Z","updated_at":"2024-01-30T16:09:30Z","cover_letter":null,"summary":null,"education_entries":[],"experience_entries":[],"skills":[],"answers":[],"resume_url":null,"tags":[],"location":{"location_str":"California, United States","country":"United States","country_co

DTO FOR POOL


In [15]:
responsespool=[]
def import_candidate_to_pool(candidates_data):
    if candidate.sourced:
        stage = "Applied"
    else:
        stage = "Sourced"
    url = f'{base_url}talent_pool/{stage}/candidates'
    
    print("Importing candidate ",{candidate.name})
    payload = candidate.to_json()
    response = requests.post(url, json=payload, headers=headers)
    # Check response status
    if response.status_code == 201:
        # print(f'Candidate {candidate.name} imported successfully!')
        print(response.text)
        # Response as JSON
        response_json = json.loads(response.text)
        # print(json.dumps(response_json, indent=4))
        
        # # Check Remaining limit
        # remaining_limit = int(response.headers.get('x-rate-limit-remaining', 0)) # If the key 'x-rate-limit-remaining' is not found in the dictionary, the get() method returns the default value, which is 0 in this case.
        # print('Remaining limit:', remaining_limit)
        print(response.status_code)
        responsespool.append(response_json)
    else:
        print(f'Failed to import candidate {candidate.name}.')
        print('Response status:', response.status_code)

print("Importing candidates...")
for candidate in candidates:
    import_candidate_to_pool(candidates)

print(responsespool)

Importing candidates...


NameError: name 'prinnt' is not defined

In [216]:
# Iterate through the list of responses
for index, response in enumerate(responsespool, start=1):
    candidate = response['candidate']
    
    # Print candidate details with numbers next to names
    print(f"{index}. Name: {candidate['name']}, Pool: {candidate['talent_pool']}")

1. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
2. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
3. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
4. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
5. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
6. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
7. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
8. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
9. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
10. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
11. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
12. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
13. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
14. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
15. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
16. Name: Uploaded Candidate, Pool: {'talent_pool_id': 636744}
1